# Training a text classifier model with fastai
- this notebook assumes you have already run text_model_training.ipynb notebook
- In this notebook, the IMDB dataset is ingested
- the first section

In [140]:
#hide
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [141]:
#hide
from fastbook import *
from fastai.text.all import *

In [142]:
# switch to control whether direct TDL or DataBlocks definition used 
tdl = True

In [143]:
modifier = 'mar3'

# Ingest the dataset
- define the path for the dataset
- create a TextDataLoaders object

In [144]:
%%time
# create dataloaders object
'''dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test', bs=16)
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)'''
# LSTM have multiple dropout probabilities for different things. Once you set them, this drop_mult property scales all of them. So you can change all dropout probabilities simultaneously using this, keeping their relative size
path = untar_data(URLs.IMDB)
path.ls()

CPU times: user 5.25 ms, sys: 0 ns, total: 5.25 ms
Wall time: 6.57 ms


(#7) [Path('/storage/data/imdb/README'),Path('/storage/data/imdb/tmp_lm'),Path('/storage/data/imdb/imdb.vocab'),Path('/storage/data/imdb/tmp_clas'),Path('/storage/data/imdb/test'),Path('/storage/data/imdb/train'),Path('/storage/data/imdb/unsup')]

# Define classifier

In [145]:
# define TextDataLoaders object
dls_clas = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')


In [146]:
# directory structure of the IMDB curated dataset
'''
├── test
│   ├── neg
│   └── pos
├── tmp_clas
├── tmp_lm
├── train
│   ├── neg
│   └── pos
└── unsup
'''

'\n├── test\n│   ├── neg\n│   └── pos\n├── tmp_clas\n├── tmp_lm\n├── train\n│   ├── neg\n│   └── pos\n└── unsup\n'

In [147]:
dls_clas.path

Path('/storage/data/imdb')

In [148]:
# save the current path
keep_path = path
print("keep_path is: ",str(keep_path))

keep_path is:  /storage/data/imdb


In [149]:
%%time
# define a text_classifier_learner object
learn_clas = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

CPU times: user 3.26 s, sys: 5.05 s, total: 8.31 s
Wall time: 2.35 s


In [150]:
# Path('/storage/data/imdb')
learn_clas.path

Path('/storage/data/imdb')

In [151]:
%%time
# set the path to the location of the encoder
learn_clas.path = Path('/notebooks/temp')

CPU times: user 0 ns, sys: 472 µs, total: 472 µs
Wall time: 68.4 µs


In [152]:
# load the encoder that was saved when the language model was trained
learn_clas = learn_clas.load_encoder('ft_'+modifier)

In [153]:
learn_clas.path

Path('/notebooks/temp')

In [154]:
# set the path back to the original path
learn_clas.path = keep_path

In [155]:
# ch 10 style Path('/storage/data/imdb')
learn_clas.path

Path('/storage/data/imdb')

In [156]:
%%time
# fine tune the model
learn_clas.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.410315,0.274882,0.887360,03:21


CPU times: user 2min 25s, sys: 55.8 s, total: 3min 21s
Wall time: 3min 21s


In [157]:
x, y = first(dls_clas.train)
x.shape, y.shape, len(dls_clas.train)

(torch.Size([64, 3345]), torch.Size([64]), 390)

In [158]:
dls_clas.show_batch()

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos xxmaj this movie was recently released on xxup dvd in the xxup us and i finally got the chance to see this hard - to - find gem . xxmaj it even came with original theatrical previews of other xxmaj italian horror classics like "" xxunk "" and "" beyond xxup the xxup darkness "" . xxmaj unfortunately , the previews were the best thing about this movie . \n\n "" zombi 3 "" in a bizarre way is actually linked to the infamous xxmaj lucio xxmaj fulci "" zombie "" franchise which began in 1979 . xxmaj similarly compared to "" zombie "" , "" zombi 3 "" consists of a threadbare plot and a handful of extremely bad actors that keeps this ' horror ' trash barely afloat . xxmaj the gore is nearly non - existent ( unless one is frightened of people running around with",neg
2,"xxbos i thought that xxup rotj was clearly the best out of the three xxmaj star xxmaj wars movies . i find it surprising that xxup rotj is considered the weakest installment in the xxmaj trilogy by many who have voted . xxmaj to me it seemed like xxup rotj was the best because it had the most profound plot , the most suspense , surprises , most xxunk the ending ) and definitely the most episodic movie . i personally like the xxmaj empire xxmaj strikes xxmaj back a lot also but i think it is slightly less good than than xxup rotj since it was slower - moving , was not as episodic , and i just did not feel as much suspense or emotion as i did with the third movie . \n\n xxmaj it also seems like to me that after reading these surprising reviews that",pos
3,"xxbos xxunk ) is the developing world 's answer to xxmaj silence of the xxmaj lambs . xxmaj where ` silence ' terrorized our peace of mind , ` citizen ' exhausts and saddens us instead . xxmaj this dramatization of the xxmaj chikatilo case translates rather well , thanks to a xxmaj westernized friendship between two xxmaj rostov cops who become equals . \n\n citizenx may also argue against ( ! ) the death penalty far better than xxmaj kevin xxmaj spacey 's xxmaj the xxmaj life of xxmaj david xxmaj xxunk ) . \n\n xxmaj humans are xxmaj machiavellian mammals , under which lie limbic brains ( lizard - logic ) . xxmaj why did two kids , who knew better , stone to death a toddler they kidnapped ? xxmaj why do bloodthirsty women yell ` li - xxunk ' at acts of xxup obscene terrorism ?",pos
4,"xxbos 8 xxmaj simple xxmaj rules for xxmaj dating xxmaj my xxmaj teenage xxmaj daughter had an auspicious start . xxmaj the supremely - talented xxmaj tom xxmaj shadyac was involved in the project . xxmaj this meant that the comedy would be nothing less of spectacular , and that 's exactly what happened : the show remains one of the freshest , funniest , wittiest shows made in a very long time . xxmaj every line , facial expression , casting choice , scene , all wreaked of perfection . xxmaj there was not one episode after which i thought , "" man that was n't as good as the rest "" . xxmaj each one was a standout . xxmaj again , this is the kind of perfectionism that we 've come to expect from xxmaj tom . xxmaj for those who do n't know , xxmaj tom",pos
5,"xxbos xxmaj warning : xxmaj spoilers xxmaj galore !

In [159]:
learn_clas.summary()

SequentialRNN (Input shape: ['64 x 3345'])
Layer (type)         Output Shape         Param #    Trainable 
LSTM                 ['64 x 33 x 1152',   1,852,416  False     
________________________________________________________________
LSTM                 ['64 x 33 x 1152',   5,317,632  False     
________________________________________________________________
LSTM                 ['64 x 33 x 400', "  1,846,400  False     
________________________________________________________________
RNNDropout           64 x 33 x 400        0          False     
________________________________________________________________
RNNDropout           64 x 33 x 1152       0          False     
________________________________________________________________
RNNDropout           64 x 33 x 1152       0          False     
________________________________________________________________
BatchNorm1d          64 x 1200            2,400      True      
_______________________________________________________

In [160]:
'''
%%time
learn_clas.freeze_to(-2)
learn_clas.fit_one_cycle(1, 2e-2)
'''

'\n%%time\nlearn_clas.freeze_to(-2)\nlearn_clas.fit_one_cycle(1, 2e-2)\n'

In [161]:
preds = learn_clas.predict("this film shows incredibly bad writing and is a complete disaster")

In [162]:
preds

('neg', TensorText(0), TensorText([0.9985, 0.0015]))

In [163]:
preds = learn_clas.predict("what a terrible film")

In [164]:
preds

('neg', TensorText(0), TensorText([0.8799, 0.1201]))

In [166]:
# save the classifier model
learn_clas.path = Path('/notebooks/temp')
learn_clas.save('classifier_single_epoch_'+modifier+'c')

Path('/notebooks/temp/models/classifier_single_epoch_mar3c.pth')